In [155]:
from DescritoresDeImagem.readwritendarray import extractimagesdescriptor, readfilendarray

# Bib joblib, Numpy,skimage,glob,re


# Standart Scale, SVC and KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report , accuracy_score, f1_score, recall_score, precision_score
from sklearn.metrics import ConfusionMatrixDisplay
from matplotlib import pyplot as plt

import numpy as np
import pickle


In [156]:
#GLCM

distances=[2,4,8,16,32,64,128,256]
angles=np.deg2rad([0,45,90,180,270])


#LBP
Winsize=25
decorr=1

CVPARTS=5 # number of CV parts
RUN=True

if RUN:
    function_glcm=extractimagesdescriptor("DadosExtraidos/Y_Resultado.txt","DadosExtraidos/X_TreinoGLCM.txt","BaseDeDados/*.bmp","GLCM",)
    function_glcm(angles=angles,distances=distances)

    function_lpq=extractimagesdescriptor(file="DadosExtraidos/Y_Resultado.txt",filename="DadosExtraidos/X_TreinoLPQ.txt",pathimg="BaseDeDados/*.bmp",descriptor="LPQ")
    function_lpq(P=25,R=6)

    function_lbp=extractimagesdescriptor(file="DadosExtraidos/Y_Resultado.txt",filename="DadosExtraidos/X_TreinoLBP.txt",pathimg="BaseDeDados/*.bmp",descriptor="LBP")
    function_lbp(winSize=Winsize,decorr=decorr)



X_lbp=readfilendarray(filename="DadosExtraidos/X_TreinoLBP.txt")

X_lpq=readfilendarray(filename="DadosExtraidos/X_TreinoLPQ.txt")

X_glcm=readfilendarray(filename="DadosExtraidos/X_TreinoGLCM.txt")

x_lpq_lpq_glcm=np.concatenate((X_lpq,X_lbp,X_glcm),axis=1)


y=readfilendarray(filename="DadosExtraidos/Y_Resultado.txt",data="s")


FUNCTION GLCM
FUNCTION LPQ
FUNCTION LBP


In [157]:
pipe=Pipeline(steps=[("scalar",StandardScaler()),
                     ("model",SVC())])


parameters={
    "model__coef0":[0,1,2,3,4,5,6],
    "model__degree":[1,2,3,4,5,6],
    "model__kernel":['linear', 'poly', 'rbf','sigmoid'],
    "model__C":[1,2,3,3.75,4,5,6],
    "model__gamma":["auto","scale"]
}

def randomgridsearchotimizator(n_iter:int,scoring:str,cvparts,pipe,parameters):
    return RandomizedSearchCV(estimator=pipe,param_distributions=parameters,n_iter=n_iter,scoring=scoring,cv=cvparts,n_jobs=-1)

svmlbp_lpq=randomgridsearchotimizator(parameters=parameters,pipe=pipe,scoring="f1_macro",cvparts=CVPARTS,n_iter=20)



In [158]:
def gridsearchotimizator(n_iter:int,scoring:str,cvparts,pipe,parameters):
    return GridSearchCV(estimator=pipe,param_grid=parameters,n_iter=n_iter,scoring=scoring,cv=cvparts,n_jobs=-1)

In [159]:
svmlbp_lpq.fit(X=x_lpq_lpq_glcm,y=y)

parameters={'model__C': 4, 'model__coef0': 1, 'model__degree': 3, 'model__gamma': 'auto', 'model__kernel': 'poly'} 
print(svmlbp_lpq.best_score_)
print(svmlbp_lpq.best_params_)

0.7634930583028159
{'model__kernel': 'poly', 'model__gamma': 'auto', 'model__degree': 6, 'model__coef0': 5, 'model__C': 1}


In [160]:
ypredsvmglcm=svmlbp_lpq.predict(X=x_lpq_lpq_glcm)

In [161]:
with open(f"Models/svm_LBP_LPQ_GLCM.pkl",mode="wb") as fsvm:
    pickle.dump(svmlbp_lpq,fsvm)

In [162]:
print("\tSVM (LBP+LPQ+GLCM) \n")
print(classification_report(y,y_pred=ypredsvmglcm)+"\n")


	SVM (LBP+LPQ+GLCM) 

              precision    recall  f1-score   support

           a       1.00      1.00      1.00       250
          ad       1.00      1.00      1.00       250
           d       1.00      1.00      1.00       250
           f       1.00      1.00      1.00       250
           j       1.00      1.00      1.00       250
          jd       1.00      1.00      1.00       250
          jt       1.00      1.00      1.00       250
           m       1.00      1.00      1.00       250
          md       1.00      1.00      1.00       250
           n       1.00      1.00      1.00       250
           o       1.00      1.00      1.00       250
           s       1.00      1.00      1.00       301

    accuracy                           1.00      3051
   macro avg       1.00      1.00      1.00      3051
weighted avg       1.00      1.00      1.00      3051




In [163]:
pipe=Pipeline(steps=[("scalar",StandardScaler()),
                     ("knn",KNeighborsClassifier())]) 
parameters={
    "knn__n_neighbors":[1,2,3,4,7,8,9],
    "knn__weights":['uniform', 'distance']
}

knnglcm=GridSearchCV(estimator=pipe,
                    param_grid=parameters,
                    scoring="f1_macro",
                    cv=CVPARTS,n_jobs=-1)


In [164]:
knnglcm.fit(X=X_glcm,y=y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scalar', StandardScaler()),
                                       ('knn', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'knn__n_neighbors': [1, 2, 3, 4, 7, 8, 9],
                         'knn__weights': ['uniform', 'distance']},
             scoring='f1_macro')

In [165]:
ypredknnglcm=knnglcm.predict(X=X_glcm)

In [166]:
with open(f"Models/svm_LBP_LPQ.pkl",mode="wb") as fsvm:
    pickle.dump(svmlbp_lpq,fsvm)
